In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [61]:
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)


In [5]:
y=mnist.train.labels

print(y)

[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]


In [26]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder("float",shape= [None ,10])

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x , W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [9]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

In [16]:
x_image = tf.reshape(x,[-1,28,28,1])
print(x_image.get_shape())

(?, 28, 28, 1)


In [19]:
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
print(h_conv1.get_shape())
h_pool1 = max_pool_2x2(h_conv1)
print(h_pool1.get_shape())

(?, 28, 28, 32)
(?, 14, 14, 32)


In [20]:
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool2 = max_pool_2x2(h_conv1)

In [21]:
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [22]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [23]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop,W_fc2)+b_fc2
y=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)


In [49]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print(correct_prediction.shape)

(?,)


In [47]:
# What does the data of MNIST look like? 
print ("What does the data of MNIST look like?")
train_images   = mnist.train.images
train_labels = mnist.train.labels
test_images    = mnist.test.images
test_labela  = mnist.test.labels
print
print (" type of 'trainimg' is %s"    % (type(trainimg)))
print (" type of 'trainlabel' is %s"  % (type(trainlabel)))
print (" type of 'testimg' is %s"     % (type(testimg)))
print (" type of 'testlabel' is %s"   % (type(testlabel)))
print (" shape of 'trainimg' is %s"   % (trainimg.shape,))
print (" shape of 'trainlabel' is %s" % (trainlabel.shape,))
print (" shape of 'testimg' is %s"    % (testimg.shape,))
print (" shape of 'testlabel' is %s"  % (testlabel.shape,))



What does the data of MNIST look like?
 type of 'trainimg' is <class 'numpy.ndarray'>
 type of 'trainlabel' is <class 'numpy.ndarray'>
 type of 'testimg' is <class 'numpy.ndarray'>
 type of 'testlabel' is <class 'numpy.ndarray'>
 shape of 'trainimg' is (55000, 784)
 shape of 'trainlabel' is (55000, 10)
 shape of 'testimg' is (10000, 784)
 shape of 'testlabel' is (10000, 10)


In [48]:
epochs_completed = 0
index_in_epoch = 0
num_examples = train_images.shape[0]

def next_batch(batch_size):
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch+=batch_size
    
    if index_in_epoch > num_examples:
        epochs_completed+=1
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = train_images[perm]
        train_labels = train_labels[perm]
        start = 0
        index_in_epoch = batch_size
        assert batch_size<= num_examples
    end = index_in_epoch
    return train_images[start:end],train_labels[start:end]

In [79]:
LEARNING_RATE=1e-4
TRAINING_ITERATIONS=2500
DROUPOUT=0.5
BATCH_SIZE=50
VALIDATION_SIZE = 2000
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

In [81]:
train_accuracies =[]
validation_accuracies =[]
x_range = []
display_step=1

for i in range(TRAINING_ITERATIONS):
    batch_xs,batch_ys = next_batch(BATCH_SIZE)
    if i%display_step == 0 or (i+1) ==TRAINING_ITERATIONS:
        train_accuracy = accuracy.eval(feed_dict={x:batch_xs,
                                                  y_:batch_ys,
                                                 keep_prob:1.0)
        if(VALIDATION_SIZE):
            validation_accuracy = accuracy.eval(feed_dict={x:validation_images[0:BATCH_SIZE],
                                                          y_:validation_labels[0:BATCH_SIZE],
                                                          keep_prob:1.0})
            print("training_accuracy / validation_accuracy => %.2f/%.2f for step %d"%(train_accuracy,validation_accuarcy,i))
            vaildation_accuraies.append(validation_accuracy)
        else :
            print("training_accuracy=> %.4f for step %d"%(train_accuracy,i))
        
        train_accuracies.append(train_accuracy)
        x_range.append(i)
        
        if i%(display_step*10)==0 and i:
            display_step*=10
            
    sess.run(train_step,feed_dict={x:batch_xs,y_:batch_ys,keep_prob:DROPOUT})
                  

InvalidArgumentError: Shape [-1,784] has negative dimensions
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda/lib/python3.6/site-packages/tornado/ioloop.py", line 888, in start